In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os.path as op
import argparse
import re
# EEG utilities
import mne
from pycrostates.cluster import ModKMeans
from pycrostates.preprocessing import extract_gfp_peaks
# BIDS utilities
from util.io.bids import DataSink
from bids import BIDSLayout

NameError: name 'BIDS_ROOT' is not defined

In [6]:
# constants
BIDS_ROOT = '../data/bids'
DERIV_ROOT = op.join(BIDS_ROOT, 'derivatives')
TASK = 'pitch'
N_MICROSTATES = 4
sub = '3'

In [14]:
# load epochs and concatenate across runs
layout = BIDSLayout(BIDS_ROOT, derivatives = True)
run = lambda f: int(re.findall('run-(\w+)_', f)[0])
fs = layout.get(
    return_type = 'filename',
    subject = sub, desc = 'forMicrostate' # Haven't successfully produced these files yet
    )
print(fs)
fs.sort(key = run)
epochs_all = [mne.read_epochs(f) for f in fs]
epochs = mne.concatenate_epochs(epochs_all)
epochs = epochs.pick('eeg')

['/project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocess_ffr/sub-3/sub-3_task-pitch_run-1_desc-forFFR_epo.fif.gz',
 '/project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocess_ffr/sub-3/sub-3_task-pitch_run-2_desc-forFFR_epo.fif.gz']

In [ ]:
# cluster observed topographies to derive microstates
peaks = extract_gfp_peaks(epochs) # topographies w/ highest signal-to-noise
ModK = ModKMeans(n_clusters = N_MICROSTATES, random_state = 0)
ModK.fit(peaks, n_jobs = -1)

fig_topos, axs = plt.subplots(1, N_MICROSTATES)
ModK.plot(axes = axs)

segmentation = ModK.predict(epochs, reject_edges = False)
parameters = segmentation.compute_parameters()

x = ModK.cluster_names
y = [parameters[elt + "_gev"] for elt in x]
fig_var, ax = plt.subplots()
sns.barplot(x = x, y = y, ax = ax)
ax.set_xlabel("Microstates")
ax.set_ylabel("Global explained Variance (ratio)")

y = [parameters[elt + "_timecov"] for elt in x]
fig_cov, ax = plt.subplots()
ax = sns.barplot(x = x, y = y, ax = ax)
ax.set_xlabel("Microstates")
ax.set_ylabel("Time Coverage (ratio)")

sink = DataSink(DERIV_ROOT, 'microstates')
solution_fpath = sink.get_path(
    subject = sub,
    task = TASK,
    desc = 'microstates',
    suffix = 'ModKMeans',
    extension = '.fif.gz'
)
ModK.save(solution_fpath)

report = mne.Report(verbose = True)
report.add_figure(
    fig_topos,
    title = 'Topographies',
    section = 'Microstates (subject-level)'
)
report.add_figure(
    fig_var,
    title = 'Variance explained',
    section = 'Microstates (subject-level)'
)
report.add_figure(
    fig_cov,
    title = 'Time coverage',
    section = 'Microstates (subject-level)'
)
report.save(op.join(sink.deriv_root, 'sub-%s.html'%sub), overwrite = True)